# Introduction to Parallelization in GRASS GIS
The goal of parallelization is to speed up computation by using multiple cores. This notebooks introduces parallelization concepts, existing parallelized tools, and approaches to parallelizing user scripts.

Let's start GRASS to run examples:

In [ ]:
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init("~/data/grassdata", "nc_basic_spm_grass7", "user1")

# Set computational region to the elevation raster.
gs.run_command("g.region", raster="elevation")

## Using already parallelized tools

There are many tools in GRASS GIS that are already parallelized, [see the list](https://grass.osgeo.org/grass-stable/manuals/keywords.html#parallel). Many tools in GRASS Addons are parallelized as well.

Generally, there are two types of implementation.
Multithreading in C tools:
   * Threads have low overhead, so they can be spawned more efficiently.
   * Tools use OpenMP API. One of the advantages of OpenMP for software distribution is that code works (runs in serial) also without OpenMP library present on the system.
   * Memory is shared, so programmer needs to be cautious about race conditions (e.g., writing into the same variable).
   
Multiprocessing in Python tools:
   * There are multiple ways to implement it, typically tools use `subprocess` and `multiprocessing` package.
   * Python tools are often wrappers around GRASS tools implemented in C. For example, tool [r.sun.daily](https://grass.osgeo.org/grass-stable/manuals/addons/r.sun.daily.html) runs [r.sun](https://grass.osgeo.org/grass-stable/manuals/r.sun.html) for multiple days in parallel.
   
Parallelized tools have `nprocs` parameter to specify number of cores to use. For C tools using OpenMP, GRASS GIS needs to be compiled with OpenMP support to take advantage of it. Both implementations work well on a single machine, but can't be scaled to a distributed system.
   
Example of calling a parallelized module in Bash and comparing the time with using 1 and 4 cores: 

In [ ]:
%%bash
time r.neighbors --q input=elevation output=elevation_smooth size=25 method=average nprocs=1
time r.neighbors --q input=elevation output=elevation_smooth size=25 method=average nprocs=4

Scaling is not linear. The speedup (processing time with 1 core / processing time with N cores) decreases when adding cores. See [benchmarks for r.neighbors](https://grass.osgeo.org/grass-stable/manuals/r.neighbors.html#performance). This behavior is due to the serial parts of the code (see [Amdahl's law](https://en.wikipedia.org/wiki/Amdahl%27s_law)) and overhead. 

## Parallelization of workflows
In a geoprocessing workflow, there are often multiple independent tasks that can be run in parallel.
The strategy how to divide the workflow into parallel tasks generally falls under data-based and task-based parallelization.
Task-based parallelism partitions tasks so that independent tasks can be completed simultaneously.
With data-based parallelization, the spatial domain is partitioned for concurrent computations of individual spatial units 
and once processed, spatial units are mosaicked back into the initial spatial domain (if applicable).

### Data-based parallelization
Data-based parallelism involves spatial domain decomposition, a process of splitting data into smaller datasets that can be processed in parallel.
As part of [GRASS GIS Python API](https://grass.osgeo.org/grass-stable/manuals/libpython/index.html), [GridModule](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass.modules.grid.html) decomposes input data into rectangular tiles, executes a given tool for each tile in parallel, and merges the results. Effectively, tiling is applied virtually, determining the spatial extent of analysis for each parallel process. In some cases such as moving window analysis, tiles need to overlap to get correct results. Note that GridModule can be fairly inefficient due to the overhead from merging results back and is therefore best suited for computationally-itensive tasks such as interpolation.

The following example shows IDW interpolation split into 4 tiles. In this case, specifying an overlap is needed to get correct results without edge artifacts. Number and size of tiles is derived from the number of cores, but can be specified.

In [ ]:
%%bash
g.region vector=elev_points res=1 -p

In [ ]:
%%python
import time
from grass.pygrass.modules.grid import GridModule
import time
start = time.time()
grid = GridModule(
    "v.surf.idw",
    input="elev_points",
    output="idw",
    column="value",
    npoints=25,
    processes=4,
    overlap=5,
    quiet=True,
)
grid.run()
print(f"Elapsed time: {time.time() - start} s")

The following is the same tool ran in serial:

In [ ]:
%%bash
time v.surf.idw --q input=elev_points column=value output=idw npoints=25

There are tools that already integrate tiling. For example, addon [r.mapcalc.tiled](https://grass.osgeo.org/grass-stable/manuals/addons/r.mapcalc.tiled.html) uses the tiling concept for  raster algebra computation. More complex algebra expression will increase the speedup of this method.

In [ ]:
%%bash
g.extension r.mapcalc.tiled
g.region raster=elevation res=1
# parallel
time r.mapcalc.tiled expression="rescaled_elevation = graph(elevation,60,1,80,10,100,100,120,100,140,1000,160,1000)" nprocs=4
# serial
time r.mapcalc expression="rescaled_elevation = graph(elevation,60,1,80,10,100,100,120,100,140,1000,160,1000)"

### Task-based parallelization
GRASS processing tools executed in parallel are separate processes that, unlike threads, do not share memory. When tasks are limited by disk I/O, parallel processing may have large overhead.


#### Examples in Python
There are multiple ways to execute tasks in parallel using Python, for example, there are libraries `multiprocessing` and `concurrent.futures`. In the following example viewsheds from different coordinates are computed in parallel using `multiprocessing.Pool` class.

To avoid issues when using multiprocessing from Jupyter Notebook, we will first write a Python file with main function and then execute it.

In [ ]:
%%writefile example.py
import os
from multiprocessing import Pool
import grass.script as gs

def viewshed(point):
    x, y, cat = point
    gs.run_command("r.viewshed", input="elevation", output=f"viewshed_{cat}", coordinates=(x, y))
    return f"viewshed_{cat}"

if __name__ == "__main__":
    viewpoints = [(633709, 225663, 1),
                  (639432, 222826, 2),
                  (640385, 220502, 3),
                  (636521, 219353, 4)]
    with Pool(processes=4) as pool:
        maps = pool.map(viewshed, viewpoints)
    print(maps)

In [ ]:
%%bash
g.region raster=elevation

In [ ]:
%run example.py

#### Examples in Bash
In a simplest case, they can be executed in parallel from a command line shell by running a geoprocessing task in the background (by appending `&`):

In [ ]:
%%bash
g.region vector=schools,firestations res=30
v.kernel --q input=schools output=kernel_schools radius=10000 &
v.kernel --q input=firestations output=kernel_firestations radius=10000 &
wait

Larger number of tasks can be scheduled to run in parallel by tools such as [GNU Parallel](https://www.gnu.org/software/parallel/) and xargs.
In this simple example, we use a loop to write commands into a file and execute those commands in parallel, using 2 cores. 
Whenever a task is finished, a next one is picked from the queued tasks.


In [ ]:
%%bash
for VECTOR in schools firestations hospitals
do
    echo v.kernel --q input=${VECTOR} output=kernel_${VECTOR} radius=10000 >> commands.sh
done
parallel -j 2 < commands.sh

See manual pages of GNU Parallel or xargs for more advanced uses. GNU Parallel can be configured to distribute jobs across multiple machines. Use `--exec` interface described below.

### Safe execution of parallel tasks

While you can execute tasks in parallel within a single mapset, it is *not safe* when your tasks:
 
 * write output maps/files with identical names (easy to fix, but common mistake)
 * modify computational region (see [solution below](#Safely-modifying-computational-region))
 * modify MASK
 * modify vector attribute database (see [solution below](#Safely-modifying-vectors-with-attributes-in-a-single-mapset))
 * use [r.reclass](https://grass.osgeo.org/grass-stable/manuals/r.reclass.html) to reclassify from the same base map

To execute tasks in separate mapsets, we can use `--exec` [interface](https://grass.osgeo.org/grass-stable/manuals/grass.html)
that allows GRASS tools and user scripts to be executed in a GRASS GIS non-interactive session.
This also enables parallelization on distributed systems.

For example, here is a simple call to list all available vectors in PERMANENT mapset:


In [ ]:
%%bash
grass ~/data/grassdata/nc_basic_spm_grass7/PERMANENT --exec g.list type=vector mapset=PERMANENT -t

One of the previous examples that was running within GRASS session in a single mapset can be rewritten so that each task runs in a newly created mapset. Note that by default newly created mapsets use default computational region for that GRASS location (use `g.region -s` to change that).

In [ ]:
%%bash
for VECTOR in schools firestations hospitals
do
    # first create a new mapset with -c flag and set computational region based on the input vector
    grass -c ~/data/grassdata/nc_basic_spm_grass7/${VECTOR} --exec g.region vector=${VECTOR} res=30
    # write the command executing v.kernel in the newly created mapset to a file
    echo grass ~/data/grassdata/nc_basic_spm_grass7/${VECTOR} --exec v.kernel --q input=${VECTOR} output=kernel_${VECTOR} radius=10000 >> exec_commands.sh
done
parallel -j 2 < exec_commands.sh

In some cases, only a temporary mapset or location is needed, see [examples](https://grass.osgeo.org/grass-stable/manuals/grass.html#batch-jobs-with-the-exec-interface).
Besides individual tools, the `--exec` interface can execute an entire script to enable more complex workflows.

#### Safely modifying computational region

Sometimes modifying computational region in a script is needed. It is a good practice to not change the global computational region, which effectively modifies a file in a mapset,
but only change the environment variable `GRASS_REGION`.
Here, we modified the previous viewshed example to compute in parallel viewsheds with different extents:

In [ ]:
%%writefile example.py
import os
from multiprocessing import Pool
import grass.script as gs

def viewshed(point):
    x, y, cat = point
    # copy current environment, modify and pass it to r.viewshed
    env = os.environ.copy()
    env["GRASS_REGION"] = gs.region_env(e=x + 300, w=x - 300, n=y + 300, s=y - 300, align="elevation")
    gs.run_command("r.viewshed", input="elevation", output=f"viewshed_{cat}", coordinates=(x, y), max_distance=300, env=env)
    return f"viewshed_{cat}"

if __name__ == "__main__":
    viewpoints = [(633709, 225663, 1),
                  (639432, 222826, 2),
                  (640385, 220502, 3),
                  (636521, 219353, 4)]
    with Pool(processes=4) as pool:
        maps = pool.map(viewshed, viewpoints)
    print(maps)

In [ ]:
%run example.py

#### Safely modifying vectors with attributes in a single mapset

By default vector maps share a single sqlite database file, however sqlite does not support concurrent write access. That poses a problem when modifying vectors with attributes in parallel. While this can be solved by running the computations in separate mapsets, it is also possible to change the default behavior to write attributes of each vector to the vector's individual sqlite file. This behavior can be activated after a new mapset is created with:

```
 db.connect driver=sqlite database='$GISDBASE/$LOCATION_NAME/$MAPSET/vector/$MAP/sqlite.db'
```